# **Ford-Sentence Classification using Naive Bayes Classifier (NBC)**

Mount the GDrive in this Google colab - 

In [1]:
# Code for mounting your Gdrive

from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


# Use text data for text classification:
Now, the Kaggle dataset - https://www.kaggle.com/datasets/gaveshjain/ford-sentence-classifiaction-dataset?resource=download, is uploaded in my GDrive and renamed the archive folder as NBC. Let's look at the contents of each file that the dataset contains.

In [5]:
import pandas as pd
# base folder name
folder_name = "/content/drive/MyDrive/NBC/"
sample_submission_csv = folder_name + "sample_submission.csv"

sample = pd.read_csv(sample_submission_csv)
# contents of sample_submission.csv
sample

,Sentence_id,Type
0,GERRES15609,Responsibility
1,PHERES15784,Responsibility
2,GERREQ10457,Requirement
3,GERSKL27235,Skill
4,HONSSK18415,SoftSkill


In [6]:
test_csv = folder_name + "test_data.csv"
test = pd.read_csv(test_csv)
# contents of test_data
test

,Unnamed: 0,Sentence_id,New_Sentence
0,0,UAERES16346,Collaborate across all of DB&T practices and o...
1,1,COGREQ15586,Strong technology expertise in Identity and A...
2,2,UAEREQ12722,Strong knowledge on Service Virtualization.
3,3,COGSKL29155,Architect scalable data processing and analyti...
4,4,PHERES12551,"Map client organization, build outstanding rel..."
...,...,...,...
15024,15024,HONEXP17084,A minimum of 3 years of experience in Industri...
15025,15025,UAESSK45553,Track record of managing multiple projects wit...
15026,15026,GERSSK14317,"In addition, the candidate should have effecti..."
15027,15027,PHEREQ4980,Candidate should have sufficient knowledge on ...


In [7]:
train_csv = folder_name + "train_data.csv"
train = pd.read_csv(train_csv)
# contents of test_data
train

,Unnamed: 0,Sentence_id,New_Sentence,Type
0,0,GERRES15609,Author and/or Review architecture/design and o...,Responsibility
1,1,PHERES15784,Should be able to develop custom dynamic shape...,Responsibility
2,2,GERREQ10457,Experience in working crosslly with a larger ...,Requirement
3,3,GERSKL27235,"Previous business experience, including but no...",Skill
4,4,HONSSK18415,Delivering fast and right the first time.,SoftSkill
...,...,...,...,...
60110,60110,UAERES18030,"In this position, you will utilize your progr...",Responsibility
60111,60111,GERRES3026,"In addition, this individual will be responsib...",Responsibility
60112,60112,INDSSK5492,Good problem solving skills.,SoftSkill
60113,60113,PHESSK15092,Good Excel knowledge .,SoftSkill


# Unique Categories/Classes:
Since we have to classify ford sentence into categories/types, let's find unique types or classes.

In [12]:
# unique categories/classes
unique_sentence_categories_type = train['Type'].unique()
for type in unique_sentence_categories_type:
  print(type)

Responsibility
Requirement
Skill
SoftSkill
Education
Experience


# **Process to build NBC:**
**Step 1:** Merge the dataset into one - test_data.csv & train_data.csv
In our case dataframes 'test' and 'train'. Used - https://pandas.pydata.org/pandas-docs/version/0.20/merging.html

In [16]:
# merge the two dataframes
dataframes = [train, test]
ford_dataset = pd.concat(dataframes)
# 15029 rows in test file + 60115 rows in train file = 75144
ford_dataset

,Unnamed: 0,Sentence_id,New_Sentence,Type
0,0,GERRES15609,Author and/or Review architecture/design and o...,Responsibility
1,1,PHERES15784,Should be able to develop custom dynamic shape...,Responsibility
2,2,GERREQ10457,Experience in working crosslly with a larger ...,Requirement
3,3,GERSKL27235,"Previous business experience, including but no...",Skill
4,4,HONSSK18415,Delivering fast and right the first time.,SoftSkill
...,...,...,...,...
15024,15024,HONEXP17084,A minimum of 3 years of experience in Industri...,NaN
15025,15025,UAESSK45553,Track record of managing multiple projects wit...,NaN
15026,15026,GERSSK14317,"In addition, the candidate should have effecti...",NaN
15027,15027,PHEREQ4980,Candidate should have sufficient knowledge on ...,NaN
